In [1]:
from collections import defaultdict
from datetime import datetime

import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.special import logit, expit
from sklearn.preprocessing import OneHotEncoder

1) Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
    - взять в тренировочный набор турниры с dateStart из 2019 года; 
    - в тестовый — турниры с dateStart из 2020 года.


In [2]:
# constants  

PLAYERS_FILE_PATH = "data/players.pkl"
TOURNAMENTS_FILE_PATH = "data/tournaments.pkl"
RESULTS_FILE_PATH = "data/results.pkl"

date_format = "%Y-%m-%dT%H:%M:%S%z"

In [3]:
with open(RESULTS_FILE_PATH, "rb") as file:
    results = pickle.load(file)  

with open(TOURNAMENTS_FILE_PATH, "rb") as file:
    tournaments = pickle.load(file)

with open(PLAYERS_FILE_PATH, "rb") as file:
    players = pickle.load(file)

Разделим все данные на train из 2019 года и test из 2020. В выборку не будем брать данные из других годов и опустим данные, в которых нет информации об ответах команд. Так же, для вопросов со "спорными" ответами ("?") и отменнеными вопросами ("Х") я произвел замену на 0 (вопрос с бесконечной сложностью, который никто не "взял"). На самом деле, я пробовал и с ними до появления комментария о сути этих обозначений, в глобальном смысле это не сильно влияет на работу модели. Еще я выкинул турниры, где заявлена всего одна команда.

In [4]:
def get_results_for_year(start_year: int):
    result = []
    for id, tour_data in results.items():
        start_date = datetime.strptime(tournaments[id]["dateStart"], date_format).year
        if start_date != start_year or len(tour_data) < 2:
            continue
        for current_team in tour_data:
            mask = current_team.get("mask", None)
            if mask:
                mask = mask.replace("?", "0").replace("X","0")
                current_team_members = current_team["teamMembers"]
                for current_member in current_team_members:
                    row = {
                        "tour_id": id,
                        "team_id": current_team["team"]["id"],
                        "player_id": current_member["player"]["id"],
                        "position": current_team.get("position", None),
                        "mask": mask,
                        "score": sum(int(x) for x in mask if x.isdigit())
                    }
                    result.append(row)
    return pd.DataFrame(result)
            

In [5]:
train_data = get_results_for_year(2019)
test_data = get_results_for_year(2020)

In [6]:
print("size of train dataset", train_data.shape[0])
print("size of test dataset", test_data.shape[0])
print("number of uniq tournaments in train dataset", len(train_data["tour_id"].unique()))
print("number of uniq tournaments in test dataset", len(test_data["tour_id"].unique()))
print("number of uniq players in train dataset", len(train_data["player_id"].unique()))
print("number of uniq players in test dataset", len(test_data["player_id"].unique()))
train_data.head(10)

size of train dataset 451776
size of test dataset 112834
number of uniq tournaments in train dataset 673
number of uniq tournaments in test dataset 172
number of uniq players in train dataset 59101
number of uniq players in test dataset 28996


,tour_id,team_id,player_id,position,mask,score
0,4772,45556,6212,1.0,111111111011111110111111111100010010,28
1,4772,45556,18332,1.0,111111111011111110111111111100010010,28
2,4772,45556,18036,1.0,111111111011111110111111111100010010,28
3,4772,45556,22799,1.0,111111111011111110111111111100010010,28
4,4772,45556,15456,1.0,111111111011111110111111111100010010,28
5,4772,45556,26089,1.0,111111111011111110111111111100010010,28
6,4772,1030,1585,5.5,111111111011110100101111011001011010,25
7,4772,1030,40840,5.5,111111111011110100101111011001011010,25
8,4772,1030,1584,5.5,111111111011110100101111011001011010,25
9,4772,1030,10998,5.5,111111111011110100101111011001011010,25


2) Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:

    - повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
    - в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
    - для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Подготовим данные для определения, на какой вопрос ответил каждый игрок

In [7]:
def prepare_data(data):
    result = defaultdict(list)
    for player_id, mask, tour_id, team_id in zip(data["player_id"], data["mask"], data["tour_id"], \
                                                           data["team_id"]):
        for index, answer in enumerate(mask):
            result["tour_id"].append(tour_id)
            result["team_id"].append(team_id)
            result["player_id"].append(player_id)
            result["question_id"].append(f"{tour_id}_{index}")
            result["answer"].append(int(answer))
    return pd.DataFrame(result)

In [8]:
train_data = prepare_data(train_data)
print(train_data.shape)
del results

(21014015, 5)


С помощью OneHotEncoder cоздадим матрицу из идентификаторов игроков и идентификаторов вопросов.

In [9]:
one_hot_encoder = OneHotEncoder(handle_unknown="ignore")
X_train = one_hot_encoder.fit_transform(train_data[["player_id", "question_id"]])
y_train = train_data["answer"]

Обучим логистическую регрессию на полученных данных

In [10]:
#logistic_regression_model = LogisticRegression(n_jobs=-1, solver="saga")
#logistic_regression_model.fit(X_train, y_train)

In [11]:
#pickle.dump(logistic_regression_model, open("model.sav", "wb"))

In [12]:
logistic_regression_model = pickle.load(open("model.sav", "rb"))

Получившиеся веса можно использовать для создания простого рейтинга игроков

In [13]:
weights = logistic_regression_model.coef_[0, :train_data.player_id.nunique()]
ids = one_hot_encoder.categories_[0]
names = [" ".join((players[x]["surname"], players[x]["name"], players[x]["patronymic"])) for x in ids]
rating = list(zip(names, weights, ids))
rating.sort(key=lambda x:x[1], reverse=True)

players_rating = pd.DataFrame(rating, columns=("name", "weight", "player_id"))
players_rating.head(50)

,name,weight,player_id
0,Руссо Максим Михайлович,4.099600,27403
1,Брутер Александра Владимировна,3.972844,4270
2,Семушин Иван Николаевич,3.945848,28751
3,Сорожкин Артём Сергеевич,3.769793,30152
4,Савченков Михаил Владимирович,3.765591,27822
5,Спешков Сергей Леонидович,3.761269,30270
6,Мереминский Станислав Григорьевич,3.619587,20691
7,Левандовский Михаил Ильич,3.616649,18036
8,Прокофьева Ирина Сергеевна,3.570724,26089
9,Николенко Сергей Игоревич,3.565426,22799


3) Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:

    - предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
    - в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


Воспользуемся достаточно сильным предположением, что игроки внутри команды отвечают независимо. При таком предположениии вероятность правильного ответа можно получить как:

$$
p_{team}(q) = 1 - \prod_{p_{player} \in team}(1-p_{player}(q))
$$


In [14]:
def get_score(data_, prediction):
    data = data_.copy()
    data["prediction"] = prediction
    data["prediction_score"] = data.groupby(["tour_id", "team_id"])["prediction"].transform(lambda x: 1 - np.prod(1 - x))
    kendall = []
    spearman = []
    for tour_id in data["tour_id"].unique():
        tours = data[data["tour_id"] == tour_id]
        spearman.append(stats.spearmanr(tours["prediction_score"], tours["score"])[0])
        kendall.append(stats.kendalltau(tours["prediction_score"], tours["score"])[0])
    return np.mean(spearman), np.mean(kendall)

In [15]:
test_data["question_id"] = "UNK"
X_test = one_hot_encoder.transform(test_data[["player_id", "question_id"]])
test_prediction_probability = logistic_regression_model.predict_proba(X_test)[:, 1]
spearman_correlation, kendall_correlation = get_score(test_data, test_prediction_probability)
print(f"Spearmen correlation: {spearman_correlation}") 
print(f"Kendall correlation: {kendall_correlation}")


Spearmen correlation: 0.787398356193356
Kendall correlation: 0.6292365393751649


Корреляции в норме

4) Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:

    - предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
    - разработайте EM-схему для обучения этой модели, реализуйте её в коде;
    - обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Имеем скрытую переменную - это вероятность правильного ответа конкретного игрока из команды. Тогда:

$$
E[z_{ij}]  =
\begin{cases} 
0, &\text{неверный ответ} \\
\frac{p(z_{ij}|\theta_n)}{1 - \prod_{p_{player} \in team}(1-p_{player}(p(z_{ij}|\theta_n))}, &\text{верный ответ}
\end{cases}
$$


Тогда, на E-шаге считаем мат. ожидание скрытой переменной, на М-шаге фиксируем скрытую переменную и обучаем модель линейной регрессии

In [16]:
def e_step(train_, prediction):
    train = train_.copy()
    train["prediction"] = prediction
    train["1 - prediction"] = train.groupby(["tour_id", "team_id"])["prediction"].transform(lambda x: 1 - np.prod(1 - x))
    z = (train["prediction"] / train["1 - prediction"]) * train["answer"]
    z = np.clip(z, 1e-5, 1-1e-5)
    return z

def m_step(X_train, X_test, z):
    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, logit(z))
    prediction = expit(model.predict(X_train))
    prediction_test = expit(model.predict(X_test))
    return prediction, get_score(test_data, prediction_test), model


In [17]:
prediction = logistic_regression_model.predict_proba(X_train)[:, 1]

for i in tqdm(range(0, 3)):
    z = e_step(train_data, prediction)
    print(f"E step of itteration {i + 1} finished")
    prediction, correlations, model = m_step(X_train, X_test, z)
    print(f"M step of ittearion {i + 1} finished")
    print(f"Spearmen correlation: {correlations[0]}") 
    print(f"Kendall correlation: {correlations[1]}")

  0%|                                                     | 0/3 [00:00<?, ?it/s]

E step of itteration 1 finished


 33%|██████████████▋                             | 1/3 [02:35<05:10, 155.06s/it]

M step of ittearion 1 finished
Spearmen correlation: 0.7880993685784247
Kendall correlation: 0.6326072880306012
E step of itteration 2 finished


 67%|█████████████████████████████▎              | 2/3 [05:16<02:38, 158.72s/it]

M step of ittearion 2 finished
Spearmen correlation: 0.7855011961865219
Kendall correlation: 0.6294422209042688
E step of itteration 3 finished


100%|████████████████████████████████████████████| 3/3 [07:53<00:00, 157.68s/it]

M step of ittearion 3 finished
Spearmen correlation: 0.7847409839352295
Kendall correlation: 0.6277698739610235


Почему-то немного скачут корреляции, не очень понимаю, почему так происходит

5) А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)?

In [18]:
weights = model.coef_[train_data.player_id.nunique():]
rating = train_data.drop_duplicates(["tour_id", "question_id"])[["tour_id"]]
rating["weight"] = weights
result = rating.groupby("tour_id").mean()
result.sort_values("weight", inplace=True)
result["name"] = [tournaments[x]["name"] for x in result.index]
result

,weight,name
tour_id,,
6149,-3.762506,Чемпионат Санкт-Петербурга. Первая лига
5928,-3.053550,Угрюмый Ёрш
5465,-3.033511,Чемпионат России
5942,-2.978832,Чемпионат Мира. Этап 2. Группа В
5159,-2.833448,Первенство правого полушария
...,...,...
5013,4.256389,(а)Синхрон-lite. Лига старта. Эпизод V
5457,4.522401,Студенческий чемпионат Калининградской области
6003,4.532996,Второй тематический турнир имени Джоуи Триббиани


Как видно из датафрейма, вверху находятся довольно крупные турниры, внизу - школьные и студенческие, в целом, соответствует интуиции